## Imports

In [ ]:
import torch
import os
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [ ]:
%run ../jsrt.py
%run ../../utils/images.py

## Load sample image

In [ ]:
from torchvision import transforms

In [ ]:
images_dir = os.path.join(DATASET_DIR, 'images')
image_fpath = os.path.join(images_dir, 'JPCLN026.png')

In [ ]:
tf = transforms.ToTensor()

In [ ]:
image = Image.open(image_fpath).convert('L')
image = tf(image)
image.size()

In [ ]:
image.mean(), image.std()

In [ ]:
plt.imshow(image[0], cmap='gray')

## Split

In [ ]:
import os
import random

In [ ]:
%run ../jsrt.py
%run ../../utils/__init__.py

In [ ]:
images_dir = os.path.join(DATASET_DIR, 'images')

metadata_fpath = os.path.join(DATASET_DIR, 'jsrt_metadata.csv')
metadata = pd.read_csv(metadata_fpath)
images = list(metadata['study_id'])
len(images), len(set(images))

In [ ]:
fold1 = [
    name
    for name in images
    if int(name[-5]) % 2 != 0
]
fold2 = [
    name for name in images
    if name not in train
]
len(fold1), len(fold2)

In [ ]:
random.shuffle(fold2)

In [ ]:
n_val = len(fold2) // 2

train = fold1
val = fold2[:n_val]
test = fold2[n_val:]
len(train), len(val), len(test)

In [ ]:
for split, images in zip(['train', 'val', 'test'], [train, val, test]):
    filepath = os.path.join(DATASET_DIR, 'splits', split)
    filepath = f'{filepath}.txt'
    write_list_to_txt(images, filepath)

## Test Dataset class

In [ ]:
%run ../jsrt.py
%run ../../utils/__init__.py

In [ ]:
dataset = JSRTDataset('train', image_size=(1024, 1024))
len(dataset)

In [ ]:
item = dataset[0]

In [ ]:
item.image.size()

In [ ]:
item.masks.size()

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.imshow(tensor_to_range01(item.image[0]), cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(item.masks)
plt.axis('off')

In [ ]:
item.masks

In [ ]:
isinstance(dataset.transform.transforms[0], transforms.Resize)

In [ ]:
isinstance(dataset.transform.transforms[1], transforms.ToTensor)

In [ ]:
dataset.transform.transforms

## Debug augmentation

### Check pil-tensor bug

In [ ]:
%run ../tools/augmentation.py

In [ ]:
dataset = JSRTDataset('train', image_size=(1024, 1024))
len(dataset)

In [ ]:
aug = Augmentator(dataset, dont_shuffle=True)
len(aug)

In [ ]:
plot_augmented_samples(aug, 0, masks=False)

### Debug augmenting seg masks

In [ ]:
%run ../common.py
%run ../../utils/__init__.py

In [ ]:
%run ../tools/augmentation.py

In [ ]:
dataset = JSRTDataset('train', image_size=(1024, 1024))
len(dataset)

In [ ]:
aug = Augmentator(dataset, dont_shuffle=True, seg_mask=True)
len(aug)

In [ ]:
plot_augmented_samples(aug, 10)

In [ ]:
masks1 = aug[10].masks
masks2 = aug[11].masks
print(masks1.min(), masks1.max(), masks1.type(), masks1.size())
print(masks2.min(), masks2.max(), masks2.type(), masks2.size())

## Compute mean and std

In [ ]:
import os
import pandas as pd

In [ ]:
%run ../jsrt.py
%run ../../utils/images.py

In [ ]:
images_dir = os.path.join(DATASET_DIR, 'images')

# metadata_fpath = os.path.join(DATASET_DIR, 'jsrt_metadata.csv')
# metadata = pd.read_csv(metadata_fpath)

# image_names = metadata['study_id']

In [ ]:
fpath = os.path.join(DATASET_DIR, 'splits', 'train.txt')
with open(fpath) as f:
    image_names = [l.strip() for l in f.readlines()]
len(image_names)

In [ ]:
iterator = ImageFolderIterator(images_dir, image_names, image_format='L')
mean, std = compute_mean_std(iterator, show=True, n_channels=1)
mean, std

## Check image sizes

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# idx = 2
# image_name = dataset.images_names[idx]

sizes = set()

for image_name in dataset.images_names:
    image_fpath = os.path.join(dataset.images_dir, image_name)

    image = Image.open(image_fpath).convert('RGB')
    sizes.add(image.size)
    
sizes

In [ ]:
plt.imshow(image)

All images are 2048 x 2048

## Compute class balance

In [ ]:
from collections import Counter
from tqdm.notebook import tqdm

In [ ]:
images_dir = os.path.join(DATASET_DIR, 'images')

image_names = os.listdir(images_dir)
len(image_names)

In [ ]:
dataset = JSRTDataset('all')
len(dataset)

In [ ]:
n_labels = len(dataset.seg_labels)

In [ ]:
counts = Counter()

for image_name in tqdm(image_names):
    masks = dataset.get_masks(image_name)
    
    counts += Counter(t.item() for t in masks.view(-1))
    
counts

In [ ]:
height, width = masks.size()
total = height * width * len(image_names)

for i_label in range(n_labels):
    amount = counts[i_label] / total * 100
    print(f'Label: {i_label}, {amount:.1f}%')